# 🏪 Walmart Weekly Sales Prediction - Neural Network

**Goal:** Build a simple Neural Network for regression to predict weekly sales.

**Dataset:** [Kaggle - Walmart Sales](https://www.kaggle.com/datasets/mikhail1681/walmart-sales)

---

## 📌 Project Overview

- **Model:** Simple Neural Network (64→32→1 neurons)
- **Framework:** TensorFlow/Keras
- **Training:** 20 epochs, batch_size=32
- **Loss:** MSE (Mean Squared Error)
- **Deployment:** Hugging Face Spaces with Gradio

---

## 1️⃣ Setup Kaggle API (Upload kaggle.json)

**Instructions:**
1. Go to your Kaggle account settings
2. Create new API token
3. Download `kaggle.json`
4. Upload it to Colab using the file upload below

In [ ]:
# Upload kaggle.json file
from google.colab import files
uploaded = files.upload()

# Setup Kaggle credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("✓ Kaggle API configured!")

## 2️⃣ Install Required Libraries

In [ ]:
%%capture
# Install libraries (suppress output)
!pip install pandas numpy scikit-learn tensorflow matplotlib joblib

## 3️⃣ Download Dataset from Kaggle

In [ ]:
# Download dataset
!kaggle datasets download -d mikhail1681/walmart-sales

# Unzip
!unzip -o walmart-sales.zip

print("\n✓ Dataset downloaded and extracted!")
!ls -lh *.csv

## 4️⃣ Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow Version: {tf.__version__}")
print("✓ Libraries imported!")

## 5️⃣ Load and Explore Data

In [ ]:
# Load dataset
df = pd.read_csv('Walmart.csv')

print("="*60)
print("DATASET OVERVIEW")
print("="*60)
print(f"Shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst 5 rows:")
display(df.head())

print(f"\nData Types:")
display(df.dtypes)

print(f"\nMissing Values:")
display(df.isnull().sum())

print(f"\nBasic Statistics:")
display(df.describe())

## 6️⃣ Data Preprocessing

In [ ]:
print("="*60)
print("DATA PREPROCESSING")
print("="*60)

# Handle missing values - fill with median
print("\n[1/4] Handling missing values...")
df.fillna(df.median(numeric_only=True), inplace=True)
print("✓ Missing values handled")

# Drop Date column (not useful for simple model)
print("\n[2/4] Dropping unnecessary columns...")
if 'Date' in df.columns:
    df = df.drop(columns=['Date'])
    print("✓ Dropped 'Date' column")

# Separate features and target
print("\n[3/4] Separating features and target...")
target_col = 'Weekly_Sales'
X = df.drop(columns=[target_col])
y = df[target_col]
print(f"✓ Features shape: {X.shape}")
print(f"✓ Target shape: {y.shape}")

# Encode categorical features
print("\n[4/4] Encoding categorical features...")
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
print(f"Categorical columns: {categorical_cols}")

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le
    
print("✓ Categorical features encoded")
print("\nFinal Features:")
display(X.head())

## 7️⃣ Train-Test Split & Scaling

In [ ]:
print("="*60)
print("TRAIN-TEST SPLIT & SCALING")
print("="*60)

# Split data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\nTraining set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# Scale features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✓ Features scaled using StandardScaler")
print(f"Feature shape: {X_train_scaled.shape[1]} features")

## 8️⃣ Build Neural Network Model

In [ ]:
print("="*60)
print("BUILDING NEURAL NETWORK")
print("="*60)

# Define model architecture
model = keras.Sequential([
    layers.Input(shape=(X_train_scaled.shape[1],)),
    layers.Dense(64, activation='relu', name='hidden_layer_1'),
    layers.Dense(32, activation='relu', name='hidden_layer_2'),
    layers.Dense(1, activation='linear', name='output_layer')
], name='Walmart_Sales_NN')

# Compile model
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

print("\nModel Architecture:")
model.summary()
print("\n✓ Model built and compiled!")

## 9️⃣ Train Model (20 Epochs)

In [ ]:
print("="*60)
print("TRAINING MODEL")
print("="*60)

# Train model
history = model.fit(
    X_train_scaled, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

print("\n✓ Training complete!")

## 🔟 Evaluate Model

In [ ]:
print("="*60)
print("MODEL EVALUATION")
print("="*60)

# Make predictions
y_pred = model.predict(X_test_scaled).flatten()

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\nTest Set Metrics:")
print("-" * 40)
print(f"MSE (Mean Squared Error):  ${mse:,.2f}")
print(f"MAE (Mean Absolute Error): ${mae:,.2f}")
print(f"RMSE (Root MSE):           ${rmse:,.2f}")
print(f"R² Score:                  {r2:.4f}")
print("-" * 40)

if r2 > 0.7:
    print("\n✅ Model performance: GOOD")
elif r2 > 0.5:
    print("\n⚠️ Model performance: MODERATE")
else:
    print("\n❌ Model performance: NEEDS IMPROVEMENT")

## 1️⃣1️⃣ Visualizations

In [ ]:
# Plot training history and predictions
plt.figure(figsize=(14, 5))

# Plot 1: Training vs Validation Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss', linewidth=2, color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2, color='orange')
plt.title('Training vs Validation Loss', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('MSE Loss', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

# Plot 2: Predicted vs Actual
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred, alpha=0.5, edgecolors='black', linewidth=0.5, s=50)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', linewidth=2, label='Perfect Prediction')
plt.title('Predicted vs Actual Sales', fontsize=14, fontweight='bold')
plt.xlabel('Actual Weekly Sales ($)', fontsize=12)
plt.ylabel('Predicted Weekly Sales ($)', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Visualizations saved as 'training_results.png'")

## 1️⃣2️⃣ Save Model & Preprocessors

In [ ]:
print("="*60)
print("SAVING MODEL & PREPROCESSORS")
print("="*60)

# Save model
model.save('walmart_sales_model.h5')
print("\n✓ Model saved: walmart_sales_model.h5")

# Save scaler
joblib.dump(scaler, 'scaler.pkl')
print("✓ Scaler saved: scaler.pkl")

# Save feature names
feature_names = X.columns.tolist()
joblib.dump(feature_names, 'feature_names.pkl')
print("✓ Feature names saved: feature_names.pkl")

# Save label encoders if any
if label_encoders:
    joblib.dump(label_encoders, 'label_encoders.pkl')
    print("✓ Label encoders saved: label_encoders.pkl")

print("\n" + "="*60)
print("✅ ALL FILES SAVED SUCCESSFULLY!")
print("="*60)

## 1️⃣3️⃣ Download Files for Deployment

In [ ]:
# Download all necessary files
from google.colab import files

print("Preparing files for download...\n")

files.download('walmart_sales_model.h5')
files.download('scaler.pkl')
files.download('feature_names.pkl')
files.download('training_results.png')

if label_encoders:
    files.download('label_encoders.pkl')

print("\n✓ Files downloaded! Ready for Hugging Face deployment.")

---

## 🤗 Hugging Face Deployment Instructions

### Step 1: Create Hugging Face Account
1. Go to [huggingface.co](https://huggingface.co)
2. Sign up for free account

### Step 2: Create New Space
1. Click **"New Space"**
2. Name: `walmart-sales-predictor`
3. SDK: **Gradio**
4. Hardware: CPU (free)

### Step 3: Upload Files
Upload these files to your Space:
- `walmart_sales_model.h5` (rename to `model.h5`)
- `scaler.pkl`
- `feature_names.pkl`
- `app.py` (create from code below)
- `requirements.txt` (create from code below)

### Step 4: Create app.py
See the next cell for the complete app.py code.

### Step 5: Create requirements.txt
```
tensorflow==2.13.0
gradio==4.19.2
joblib==1.3.2
numpy==1.24.3
```

### Step 6: Deploy!
Your app will be live at: `https://huggingface.co/spaces/YOUR_USERNAME/walmart-sales-predictor`

---

## 📝 app.py Code for Hugging Face

In [ ]:
# This is the app.py code - copy this to your Hugging Face Space

app_code = '''
import gradio as gr
import numpy as np
import joblib
import tensorflow as tf
from tensorflow import keras

# Load model and preprocessors
model = keras.models.load_model('model.h5')
scaler = joblib.load('scaler.pkl')
feature_names = joblib.load('feature_names.pkl')

def predict_sales(*inputs):
    try:
        input_data = np.array([list(inputs)])
        input_scaled = scaler.transform(input_data)
        prediction = model.predict(input_scaled, verbose=0)[0][0]
        return f"💰 **Predicted Weekly Sales: ${prediction:,.2f}**"
    except Exception as e:
        return f"❌ Error: {str(e)}"

# Create inputs
inputs = [gr.Number(label=feature, value=0) for feature in feature_names]

# Create interface
demo = gr.Interface(
    fn=predict_sales,
    inputs=inputs,
    outputs=gr.Textbox(label="Prediction", lines=2),
    title="🏪 Walmart Weekly Sales Predictor",
    description="Simple Neural Network Regression Model",
    theme=gr.themes.Soft()
)

demo.launch()
'''

print("Copy the code above to create app.py in your Hugging Face Space")
print("\nOr download it:")

with open('app.py', 'w') as f:
    f.write(app_code)
    
files.download('app.py')

## 📋 requirements.txt Code

In [ ]:
requirements = '''tensorflow==2.13.0
gradio==4.19.2
joblib==1.3.2
numpy==1.24.3
'''

with open('requirements.txt', 'w') as f:
    f.write(requirements)
    
print("requirements.txt created!")
files.download('requirements.txt')

---

## ✅ PROJECT COMPLETE!

### What You Have:
- ✅ Trained Neural Network model
- ✅ Model evaluation metrics
- ✅ Training visualizations
- ✅ Saved model files
- ✅ Deployment code (app.py)
- ✅ Requirements file

### Next Steps:
1. Download all files from this notebook
2. Create Hugging Face Space
3. Upload files to Space
4. Deploy and share!

### For Local Deployment:
1. Save all files to a local folder
2. Install requirements: `pip install -r requirements.txt`
3. Run: `python app.py`
4. Open: `http://localhost:7860`

---

**Made with ❤️ for academic submission**

**Date:** 2026-02-13